In [1]:
import random
import numpy as np
import pandas as pd

# dados linearmente separáveis
df0 = pd.DataFrame([[0, 0, 0, 0], 
                    [1, 0, 0, 0],
                    [1, 1, 0, 1],
                    [1, 0, 1, 1],
                    [1, 1, 1, 1]],
                    columns = ['x0', 'x1', 'x2', 'y'])

# dados linearmente separáveis
df1 = pd.DataFrame([[0, 0, 0, 0, 0],
                    [0, 0, 1, 0, 0],
                    [1, 0, 0, 0, 0],
                    [1, 1, 0, 0, 0],
                    [1, 0, 1, 0, 0],
                    [1, 1, 1, 0, 1],
                    [1, 0, 1, 1, 1],
                    [1, 1, 1, 1, 1]],
                    columns = ['x0', 'x1', 'x2', 'x3', 'y'])

# dados não linearmente separáveis
df2 = pd.DataFrame([[0, 0, 0, 1],
                    [0, 1, 0, 0],
                    [0, 0, 1, 0],
                    [1, 0, 1, 0],
                    [1, 1, 1, 1]],
                    columns = ['x0', 'x1', 'x2', 'y'])
# semente
seed = 123
random.seed(seed)

In [2]:
def get_errors(feature_matrix, label, coefficients, threshold, upper, lower): 
    
    count = 0
    pred_dict = {}
    for n in range(len(coefficients)): # calculando e limiarizando predições 
        predicted = round(sum(feature_matrix.iloc[n] * coefficients))
        
        if predicted >= threshold:
            predicted = upper
        else:
            predicted = lower
        
        error_ids = []
        pred_dict[count] = {'predicted' : predicted, 'actual' : label.iloc[n]}
        count += 1
        
    for key in pred_dict.keys(): # comparando predições com valores verdadeiros
        if pred_dict[key]['predicted'] != pred_dict[key]['actual']:
            error_ids.append(key)  
            
    if len(error_ids) == 0:                       # condição de parada (convergência)
        return None, coefficients, pred_dict      
    else:
        return min(error_ids), coefficients, pred_dict 
    # caso haja erro, retorna por onde deve começar a 'revisitar'

def coefficient_optimization(error_id, coef_list, pred_dict, learning_rate, feature_matrix, label): 
    # 'revisita' (na primeira predição errada)
    if error_id == None: # interpreta condição de parada implementada na função anterior
        return 'stop', coef_list

    bad_row = feature_matrix.iloc[error_id] # otimiza coeficiente, à partir da observação errada
    new_coefs = []
    for n in range(len(bad_row)):#'bad_row' é uma 'series', aqui, acesso cada variável explicativa da observação
        new_coefs.append(coef_list[n] + learning_rate * bad_row[n] * \
                         (label.iloc[error_id] - pred_dict[error_id]['predicted']))    
        
    return 'go', new_coefs

# esta função executa as duas anteriores e interpreta suas saídas, implementa condição de parada
def perceptron(feature_matrix, label, coefficients, learning_rate, threshold, upper, lower, max_epoch):
    
    run = 'go'
    epoch = 0
    while True:
        if run == 'go':
            
            error_id, coef_list, pred_dict = get_errors(feature_matrix, label, coefficients, 
                                                        threshold, upper, lower)
            
            run, new_coefs = coefficient_optimization(error_id, coef_list, pred_dict,
                                                     learning_rate, feature_matrix, label) 
            coefficients = new_coefs
            epoch += 1

            if epoch == max_epoch and run != 'stop':
                return new_coefs, 'max_epoch reached, no convergence'

            elif epoch != max_epoch and run == 'stop':  
                return new_coefs, 'convergence achieved'

            elif epoch == max_epoch and run == 'stop':
                return new_coefs, 'max_epoch reached, convergence achieved'

In [3]:
# ajuste a df0

# coeficientes de df0 não otimizados
coefficients0 = random.choices(population = np.arange(start = -1, stop = 1, step = 0.001), k = 3)
print(f'previous coefficients: {coefficients0}')
# rodando em dados linearmente separáveis
coefficients0 = random.choices(population = np.arange(start = -1, stop = 1, step = 0.001), k = 3)
coefs0, report0 = perceptron(coefficients = coefficients0, threshold = 0.5, upper = 1, lower = 0,
                            learning_rate = 0.1, feature_matrix = df0[['x0', 'x1', 'x2']], label = df0['y'],
                            max_epoch = 25)
# coeficientes de df0 otimizados
print(f'updated coefficients:  {coefs0}')
print(report0)

previous coefficients: [-0.8959999999999999, -0.8259999999999998, -0.18599999999999928]
updated coefficients:  [-0.4849999999999999, 1.1020000000000016, -0.9239999999999999]
convergence achieved


In [4]:
# ajuste a df1

# coeficientes de df1 não otimizados
coefficients1 = random.choices(population = np.arange(start = -1, stop = 1, step = 0.001), k = 4)
print(f'previous coefficients: {coefficients1}')
# rodando em dados linearmente separáveis
coefs1, report1 = perceptron(coefficients = coefficients1, threshold = 0.5, upper = 1, lower = 0, 
                             learning_rate = 0.1, feature_matrix = df1[['x0', 'x1', 'x2', 'x3']], 
                             label = df1['y'], max_epoch = 25)
# coeficientes de df1 otimizados
print(f'updated coefficients:  {coefs1}')
print(report1)

previous coefficients: [0.07200000000000095, -0.3359999999999994, 0.7040000000000015, -0.6809999999999997]
updated coefficients:  [0.07200000000000095, -0.3359999999999994, 0.4040000000000016, -0.6809999999999997]
convergence achieved


In [5]:
# ajuste a df2

# coeficientes de df2 não otimizados
coefficients2 = random.choices(population = np.arange(start = -1, stop = 1, step = 0.001), k = 3)
print(f'previous coefficients: {coefficients2}')
# rodando em dados não linearmente separáveis
coefs2, report2 = perceptron(coefficients = coefficients2, threshold = 0.5, upper = 1, lower = 0, 
                             learning_rate = 0.1, feature_matrix = df2[['x0', 'x1', 'x2']], label = df2['y'],
                             max_epoch = 25)
# coeficientes de df2 'otimizados'
print(f'updated coefficients:  {coefs2}')
print(report2)

previous coefficients: [-0.3259999999999994, -0.3329999999999994, -0.5099999999999996]
updated coefficients:  [-0.3259999999999994, -0.3329999999999994, -0.5099999999999996]
max_epoch reached, no convergence
